In [ ]:
%matplotlib inline
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from polyphys.manage import organizer
from polyphys.manage import parser
from polyphys.visualize import tuner as ptuner
from polyphys.visualize import plotter
import seaborn as sns

## Excluded-volume models

### Loading data
- Creating *allInOne* dataset: Do this step if there is no  **allInOne-vExc.csv** file, or it exist but has to bew updated.

In [ ]:
#vexc_files = glob("/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Mathematica/vExc_Models/vExc_data/*.csv")
vexc_files = glob("/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Mathematica/vExc_Models/vExc_data/*.csv")
vexc_files = organizer.sort_filenames(vexc_files, fmts=['csv'])
vexc_df = []
limit = True # limit exclude volumes to --vexc_ath, vexc_athr]
for vexc_file in vexc_files:
    vexc_info = parser.ExcludedVolume(vexc_file[0])
    vexc_info.read_data()
    vexc_info.scale(limit=limit)
    vexc_info.add_model_info()
    vexc_df.append(vexc_info.vexc_df)
vexc_df = pd.concat(vexc_df, axis=0)
vexc_df.reset_index(inplace=True, drop=True)
if limit is True:
    vexc_df.to_csv('allInOne-vExc-limit.csv', index=False)
else:
    vexc_df.to_csv('allInOne-vExc.csv', index=False)

- DO this if you want to use an existing **allInOne-vExc.csv** file

In [ ]:
vexc_df = pd.read_csv('allInOne-vExc-limit.csv')

### Plots

In [ ]:
dcrowds = [0.2, 0.3, 0.6, 0.8, 1.0, 2.0, 4.0, 6.0]
plotter.p_vexc_models(
    vexc_df,
    dcrowds
)

In [ ]:
label_dict_long = {"vfrc_c":r"Volume fraction of crowders ($\phi_c$)",
             "vfrc_c_scaled":r"Rescaled volume fraction of crowders (${a\phi_c}/{a_c}$)",
             "exc_vol_scaled":r"Rescaled exc. volume ($v_{exc}/v_{athr}$)",
                  "r_scaled_max":r"Normalized chain size ($R/R_{0}$)",
                  "rho_c":r"Density of crowders ($\rho_c$) in $a^{-3}$"}

In [ ]:
dcrowds = [0.2, 0.8, 1.0, 2.0, 5.0]
plotter.p_vexc_dcrowds(
    vexc_df,
    dcrowds,
    fontsize=16
)

## Free-energy approach

### Loading data

In [ ]:
#vexc_files = glob("/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Mathematica/vExc_Models/vExc_data/*.csv")
#free_energy_files = glob("/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Mathematica/Free energy approach/FreeEnergyApproach-VirialExpansion-allInOne-data/*.csv")
free_energy_files = glob('/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Mathematica/Free energy approach/FreeEnergyApproach-VirialExpansion-allInOne-data/*.csv')
free_energy_files = organizer.sort_filenames(free_energy_files, fmts=['csv'])
phi_c_out_cap=0.45
chain_free_energy = []
for free_energy_file in free_energy_files[:2]:
    free_energy_info = parser.FreeEnergyVirial(free_energy_file[0])
    free_energy_info.read_data()
    free_energy_info.scale(phi_c_out_cap)
    free_energy_info.add_model_info()
    free_energy_info.r_chain_df.sort_values(by='phi_c_out')
    chain_free_energy.append(free_energy_info.r_chain_df)
chain_free_energy = pd.concat(chain_free_energy, axis=0)
chain_free_energy.reset_index(inplace=True, drop=True)
chain_free_energy.to_csv(
    f'allInOne-freeEnergyVirial-phiCrdLimit{phi_c_out_cap}.csv',
    index=False
)
mathematica_cols = [
    'rhoOut', 'rhoIn', 'rChain', 'phiCout', 'phiCin','rScaled','tailModel',
    'vdepModel', 'nMon', 'dCyl', 'dCrowd'
]
chain_free_energys_mathemaitca = chain_free_energy.copy()
chain_free_energys_mathemaitca.columns = mathematica_cols
chain_free_energys_mathemaitca.to_csv(
    f'allInOne-freeEnergyVirial-phiCrdLimit{phi_c_out_cap}-Mathemaitca.csv',
    index=False
)
del chain_free_energys_mathemaitca

In [ ]:
## Extendeding data set for Mathematica
free_energy_files = glob("/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Mathematica/Free energy approach/FreeEnergyApproach-VirialExpansion-allInOne-data/*.csv")
free_energy_files = organizer.sort_filenames(free_energy_files, fmts=['csv'])
phi_c_out_cap=0.45
for free_energy_file in free_energy_files:
    free_energy_info = parser.FreeEnergyVirial(free_energy_file[0])
    free_energy_info.read_data()
    free_energy_info.scale(phi_c_out_cap)
    free_energy_info.add_model_info()
    free_energy_info.r_chain_df.sort_values(by='phi_c_out')
    free_energy_info.r_chain_df.loc[:,'phi_c_relative'] = free_energy_info.r_chain_df.loc[:, 'phi_c_in'] / free_energy_info.r_chain_df.loc[:, 'phi_c_out']
    free_energy_info.r_chain_df.loc[:,'confinement_rate'] = free_energy_info.r_chain_df.loc[:, 'dcrowd'] / free_energy_info.r_chain_df.loc[:,'dcyl']
    free_energy_info.r_chain_df.loc[:,'confinement_rate'] = free_energy_info.r_chain_df.loc[:,'confinement_rate'].round(decimals=3)
    free_energy_info.r_chain_df.loc[:,'phi_c_out_scaled'] = free_energy_info.r_chain_df.loc[:, 'phi_c_out'] / free_energy_info.r_chain_df.loc[:, 'dcrowd']
    free_energy_info.r_chain_df.to_csv(
        "./free_energy_extended/"+free_energy_file[0].split("/")[-1],
        index=False,
        header=False
    )

In [ ]:
free_energy_file[0].split("/")[-1]

- DO this if you want to use an existing **allInOne-vExc.csv** file

In [ ]:
phi_c_out_cap=0.45
chain_free_energy_original = pd.read_csv(f'allInOne-freeEnergyVirial-phiCrdLimit{phi_c_out_cap}.csv')

### finding the uppoer limit on $\rho_o$ based on the limit on $\phi_o$

In [ ]:
phi_c_out_cap=0.45
chain_free_energy.groupby(['nmon', 'dcyl', 'dcrowd']).max().reset_index()[['nmon', 'dcyl', 'dcrowd','rho_c_out']].to_csv(f"max_rho_c_out_for_phi_c_limit{phi_c_out_cap}.csv", index =False)

## Viz

In [ ]:
dcrowds = [0.2, 0.3, 0.6, 1, 2, 4, 6]
dcyls = [5, 15, 30]
nmons = [80, 500, 2000]
cond = (
    chain_free_energy_original['nmon'].isin(nmons) &
    chain_free_energy_original['dcrowd'].isin(dcrowds) &
    chain_free_energy_original['dcyl'].isin(dcyls)
)
chain_free_energy = chain_free_energy_original[cond].copy()
chain_free_energy

In [ ]:
chain_free_energy.loc[:,'phi_c_relative'] = chain_free_energy.loc[:, 'phi_c_in'] / chain_free_energy.loc[:, 'phi_c_out']
chain_free_energy.loc[:,'confinement_rate'] = chain_free_energy.loc[:, 'dcrowd'] / chain_free_energy.loc[:,'dcyl']
chain_free_energy.loc[:,'confinement_rate'] = chain_free_energy.loc[:,'confinement_rate'].round(decimals=3)
chain_free_energy.loc[:,'phi_c_out_scaled'] = chain_free_energy.loc[:, 'phi_c_out'] / chain_free_energy.loc[:, 'dcrowd']

In [ ]:
pd.set_option('display.max_columns', 75)
chain_free_energy[['dcrowd','dcyl','confinement_rate']].sort_values(by=['confinement_rate']).drop_duplicates()

In [ ]:
flar_cmap = mpl.colors.ListedColormap(sns.cm._flare_lut)
flare_cmap_cut = ptuner.truncated_colormap(flar_cmap,  min_value=0.0, max_value=1.0, ncolors=200)
mpl.cm.register_cmap("flare_cmap_cut", flare_cmap_cut)

In [ ]:
conf_rates = chain_free_energy['confinement_rate'].drop_duplicates().sort_values().values
conf_rate_colors = sns.color_palette("coolwarm_r", len(conf_rates))#,as_cmap=True)
conf_rate_dict = dict(zip(conf_rates,conf_rate_colors))
conf_rate_colors

In [ ]:
cond_ha = chain_free_energy['vdep_model']=='Ha'
chain_free_energy_ha = chain_free_energy.loc[cond_ha,:]

In [ ]:
tail_fullname = [' | LJ power-6 law', 
                 ' | Three-body term']

sns.plotting_context(font_scale=3)
#sns.set_style("white")
sns.axes_style("white")
chain_grid = sns.relplot(
    data=chain_free_energy_ha,
    x='phi_c_out',
    y='r_scaled',
    hue='confinement_rate',
    style='dcyl',
    size='dcrowd',
    col='tail_model',
    row='nmon',
    kind='scatter',
    palette=conf_rate_dict,
    height=4,
    alpha=None
) 
#chain_grid.set_titles(template=r"$N={row_name}$ | 'Monomer-monomer interaction:{col_name}")

for row in range(len(chain_grid.axes)):
    chain_grid.axes[row,0].set_title(fr"$N={nmons[row]}$" + tail_fullname[0])
    chain_grid.axes[row,0].grid(True, ls=':', lw=1)
    chain_grid.axes[row,0].tick_params(axis='both', direction='inout', width=1.25)
    chain_grid.axes[row,1].set_title(fr"$N={nmons[row]}$" + tail_fullname[1])
    chain_grid.axes[row,1].grid(True, ls=':', lw=1)
    chain_grid.axes[row,1].tick_params(axis='both', direction='inout', width=1.25)
chain_grid._legend.texts[0].set_text(r"${a_c}/{D}$ (color)")
chain_grid._legend.texts[1+len(conf_rates)].set_text(r"${a_c}/{a}$")
chain_grid._legend.texts[2+len(conf_rates)+len(dcrowds)].set_text(r"${D}/{a}$")
chain_grid.set_ylabels(r"${R}/{R_0}$")
chain_grid.set_xlabels(r"$\phi_c^{(out)}$")
chain_grid.tight_layout()

In [ ]:
tail_fullname = [' | LJ power-6 law', 
                 ' | Three-body term']

sns.plotting_context(font_scale=3)
#sns.set_style("white")
sns.axes_style("white")
chain_grid = sns.relplot(
    data=chain_free_energy_ha,
    x='phi_c_out',
    y='phi_c_relative',
    hue='confinement_rate',
    style='dcyl',
    size='dcrowd',
    col='tail_model',
    row='nmon',
    kind='scatter',
    palette=conf_rate_dict,
    height=4,
    alpha=None
) 
#chain_grid.set_titles(template=r"$N={row_name}$ | 'Monomer-monomer interaction:{col_name}")

for row in range(len(chain_grid.axes)):
    chain_grid.axes[row,0].set_title(fr"$N={nmons[row]}$" + tail_fullname[0])
    chain_grid.axes[row,0].grid(True, ls=':', lw=1)
    chain_grid.axes[row,0].tick_params(axis='both', direction='inout', width=1.25)
    chain_grid.axes[row,1].set_title(fr"$N={nmons[row]}$" + tail_fullname[1])
    chain_grid.axes[row,1].grid(True, ls=':', lw=1)
    chain_grid.axes[row,1].tick_params(axis='both', direction='inout', width=1.25)
chain_grid._legend.texts[0].set_text(r"${a_c}/{D}$ (color)")
chain_grid._legend.texts[1+len(conf_rates)].set_text(r"${a_c}/{a}$")
chain_grid._legend.texts[2+len(conf_rates)+len(dcrowds)].set_text(r"${D}/{a}$")
chain_grid.set_ylabels(r"${\phi_c^{(in)}}/{\phi_c^{(out)}}$")
chain_grid.set_xlabels(r"$\phi_c^{(out)}$")
chain_grid.tight_layout()

In [ ]:
sns.relplot(
    data=chain_free_energy,
    x='phi_c_out',
    y='r_scaled',
    hue='dcyl',
    style='dcrowd',
    col='nmon',
    kind='scatter',
    palette="Spectral",
    col_wrap=3
)

In [ ]:
sns.relplot(
    data=chain_free_energy,
    x='phi_c_out',
    y='r_scaled',
    hue='nmon',
    style='dcyl',
    col='dcrowd',
    kind='scatter',
    palette="Spectral",
    col_wrap=3
)

In [ ]:
sns.relplot(
    data=chain_free_energy,
    x='phi_c_out',
    y='r_scaled',
    hue='dcyl',
    style='nmon',
    col='dcrowd',
    kind='scatter',
    palette="Spectral",
    col_wrap=3
)